In [11]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from random import randint
from scipy.signal import convolve2d

def planck_law(wavelength, temperature):
    h = 6.626e-34
    c = 3.0e+8
    k = 1.38e-23
    conditions = [wavelength * k * temperature != 0]
    functions = [lambda x, temp: (2 * h * c ** 2) / (x ** 5 * (np.exp((h * c) / (x * k * temp)) - 1))]
    return np.piecewise(wavelength, conditions, functions, 0, (temperature,))


def generate_star_temperature():
    return randint(3000, 10000)

def generate_star_field(temperatures, wavelength):
    x = np.linspace(0, 1, 100)
    y = np.linspace(0, 1, 100)
    X, Y = np.meshgrid(x, y)

    Z = np.zeros((100, 100))
    for temperature in temperatures:
        intensity = planck_law(wavelength * 1e-9, temperature)
        max_intensity = np.max(planck_law(np.linspace(1e-9, 3e-6, 1000), temperature))
        Z += intensity / max_intensity

    Z /= len(temperatures)
    return Z

def airy_disk_psf(size, radius):
    x, y = np.meshgrid(np.arange(-size//2, size//2 + 1), np.arange(-size//2, size//2 + 1))
    r = np.sqrt(x**2 + y**2)
    sinc_term = 2 * np.sinc(radius * r)
    psf = np.where(r != 0, (sinc_term / (radius * r)) ** 2, 1)
    return psf / np.sum(psf)

def main():
    num_stars = randint(5, 50)
    wavelengths = [500]  # Choose one wavelength to display the image

    temperatures = [generate_star_temperature() for _ in range(num_stars)]

    for wavelength in wavelengths:
        star_field = generate_star_field(temperatures, wavelength)

        # Apply Airy disk PSF
        psf_size = 11
        psf_radius = 1.0
        psf = airy_disk_psf(psf_size, psf_radius)
        star_field_blurred = convolve2d(star_field, psf, mode='same')

        # Display the star field image
        plt.imshow(star_field_blurred, cmap='gray', origin='lower')
        plt.title(f"Synthetic Star Field at {wavelength}nm")
        plt.show()

if __name__ == "__main__":
    main()


TypeError: <lambda>() takes 2 positional arguments but 3 were given